# **Data Linkage using of Record Linkage and Fuzzy Matching**

This notebook goes through the data linkage of authors across two provided data sources using both Record Linkage and Fuzzy Matching, as well as concluding their precision, recall and F1- meansure.

In [2]:
!pip install fuzzymatcher

     |████████████████████████████████| 50 kB 2.9 MB/s 
  Created wheel for metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13919 sha256=5ee5f4271ed1226143309410b5daadfbe8f5b38c3490b526f890f60a7525e65d
  Stored in directory: /root/.cache/pip/wheels/1d/a8/cb/6f8902aa5457bd71344e00665c230e9c45255b3f57f2194a0f
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149868 sha256=5007a7ccfc66520c6d61e39f7e2ff1d16ae016884c445d3061a067df7d00041b
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built metaphone python-Levenshtein


In [3]:
!pip install recordlinkage

     |████████████████████████████████| 944 kB 5.9 MB/s 
     |████████████████████████████████| 137 kB 41.8 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.8.9-cp37-cp37m-linux_x86_64.whl size=73224 sha256=41876d3a325ed56ac64db8d0ec2822a434ec592b30c95d5bef02671054f88aa6
  Stored in directory: /root/.cache/pip/wheels/32/a9/ef/5d8742e72deaf0d1de327a180d008c2c0299367581800ea73f
Successfully built jellyfish


In [4]:
import pandas as pd
from pathlib import Path
import fuzzymatcher
import recordlinkage

# **Utilizing FuzzyMatcher** 

This section involves performing performing fuzzy matching on the two data sets and finally comparing with the ground truth set provided to confirm  precision, recall and F1- meansure.

In [5]:
ACM = pd.read_csv( 'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/ACM.csv')
DBLP = pd.read_csv( 'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/dblp.csv')

In [6]:
## rename columns of the dataset for personal convenience
ACM.rename(columns = {'id':'ACM_id', 'title':'ACM_title', 'authors':'ACM_authors',
                              'venue':'ACM_venue', 'year':'ACM_year'}, inplace = True)
ACM.head()

,Unnamed: 0,ACM_id,ACM_title,ACM_authors,ACM_venue,ACM_year
0,0,304586,The WASA2 object-oriented workflow management ...,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,1,304587,A user-centered interface for querying distrib...,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
2,2,304589,"World Wide Database-integrating the Web, CORBA...","Athman Bouguettaya, Boualem Benatallah, Lily H...",International Conference on Management of Data,1999
3,3,304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...",International Conference on Management of Data,1999
4,4,304582,The CCUBE constraint object-oriented database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...",International Conference on Management of Data,1999


In [7]:
DBLP.rename(columns = {'id':'DBLP_id', 'title':'DBLP_title', 'authors':'DBLP_authors',
                              'venue':'DBLP_venue', 'year':'DBLP_year'}, inplace = True)
DBLP.head()

,Unnamed: 0,DBLP_id,DBLP_title,DBLP_authors,DBLP_venue,DBLP_year
0,0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models f...,D. Scott Mackay,SIGMOD Record,1999
1,1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and it...,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
2,2,conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive A...,"Themistoklis Palpanas, Richard Sidle, Hamid Pi...",VLDB,2002
3,3,conf/vldb/GardarinGT96,Cost-based Selection of Path Expression Proces...,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert G...",VLDB,1996
4,4,conf/vldb/HoelS95,Benchmarking Spatial Join Operations with Spat...,"Erik G. Hoel, Hanan Samet",VLDB,1995


In [8]:
## printing the dataframe sizes

print("raw dataframe number of records:")
print("ACM_id count:  {x}".format(x=ACM['ACM_id'].count()))
print("DBLP_id count:  {x}".format(x=DBLP['DBLP_id'].count()))

raw dataframe number of records:
ACM_id count:  2294
DBLP_id count:  2616


# **Choosing to focus on "ACM_authors" and "ACM_title and comparing the top few rows of the data"**

In [9]:
cond = ACM['ACM_authors'].str.startswith('A')
ACM.sort_values(by=['ACM_authors'], ascending=True)[['ACM_authors', 'ACM_title']].head(20)

,ACM_authors,ACM_title
2135,Objectivity,Objectivity Industrial Exhibit
725,The TimesTen Team,Mid-tier caching: the TimesTen approach
1614,&#214;zg&#252;r Ulusoy,An annotated bibliography on real-time databas...
376,"&#214;zg&#252;r Ulusoy, Alejandro Buchmann",Exploiting main memory DBMS features to improv...
1017,"&#216;ystein Gr&#248;vlen, Svein-Olaf Hvasshov...",Low-Cost Compensation-Based Query Processing
1054,"A. A. Diwan, Sanjeeva Rane, S. Seshadri, S. Su...",Clustering Techniques for Minimizing External ...
1741,"A. Biliris, S. Dar, N. Gehani, H. V. Jagadish,...",ASSET: a system for supporting extended transa...
235,"A. Biliris, T. A. Funkhouser, W. O'Connell, E....",BeSS: storage support for interactive visualiz...
1565,"A. Dogac, C. Dengi, E. Kilic, G. Ozhan, F. Ozc...",METU interoperable database system
218,"A. Formica, H. D. Groger, M. Missikoff",An efficient method for checking object-orient...


In [10]:
cond = DBLP['DBLP_authors'].str.startswith('A')
DBLP.sort_values(by=['DBLP_authors'], ascending=True)[['DBLP_authors', 'DBLP_title']].head(20)

,DBLP_authors,DBLP_title
1888,?,"Title, Announcement"
1913,?,"Title, Foreword"
1925,?,"The Active Database Central, ER2000, VLDB 2000..."
1946,?,"Title, Preface to the Special Issue on Persist..."
1873,?,"Title, Foreword"
2458,?,"Title, Foreword, Special Issue Editorial"
666,?,"Title, Foreword, Preface to the Special Issue ..."
451,?,Calls for Papers / Announcements
2437,?,Title
454,?,Proceedings of the 2003 ACM SIGMOD Internation...


In [11]:
## defining columns for the left and right of the DataFrame
left_on = ["ACM_authors", "ACM_title", "ACM_venue", "ACM_year"]
right_on = ["DBLP_authors", "DBLP_title", "DBLP_venue", "DBLP_year"]

# **Using the Fuzzymatcher to attempt to Find matches**

utilizing the fuzzy_left_join method in order to uncover matches among the two data sets

In [12]:
matched_results = fuzzymatcher.fuzzy_left_join(ACM,
                                            DBLP,
                                            left_on,
                                            right_on,
                                            left_id_col='ACM_id',
                                            right_id_col='DBLP_id')

In [67]:
## Reorder and display the best matches in a more readable format
cols = [
    "best_match_score", "ACM_authors", "DBLP_authors", "ACM_venue", "DBLP_venue",
    "ACM_year", "DBLP_year","ACM_id","DBLP_id"
]
pd.set_option('display.max_columns', None)
compression_opts = dict(method='zip',
                        archive_name='matched_results.csv')  
matched_results.to_csv('matched_results.zip', index=False,
          compression=compression_opts)
matched_results[cols].sort_values(by=['best_match_score'], ascending=False).head(15)


,best_match_score,ACM_authors,DBLP_authors,ACM_venue,DBLP_venue,ACM_year,DBLP_year,ACM_id,DBLP_id
25282,9.284360,"Michael J. Carey, Steve Kirsch, Mary Roth, Ber...","Runping Qi, Kuan Yee, Brian Sterling, Nicolas ...",Very Large Data Bases,VLDB,2001,2001,672199,conf/vldb/CareyKRLABFLOPQRSSUVWY01
34637,9.233475,"Eugene Inseok Chong, Jagannathan Srinivasan, S...","Mahesh Jagannath, Eugene Inseok Chong, Aravind...",ACM SIGMOD Record,SIGMOD Record,2003,2003,777000,journals/sigmod/ChongFTSYKDJJ03
29328,9.165616,"Richard Thomas Snodgrass, Ilsoo Ahn, Gadi Aria...","Wolfgang Käfer, Curtis E. Dyreson, Arie Segev,...",ACM SIGMOD Record,SIGMOD Record,1994,1994,181562,journals/sigmod/SnodgrassAABCDEGJKKKLLRSSS94
28664,9.091821,"Richard Thomas Snodgrass, Ilsoo Ahn, Gad Ariav...","James Clifford, Ramez Elmasri, Michael D. Soo,...",ACM SIGMOD Record,SIGMOD Record,1994,1994,187449,journals/sigmod/SnodgrassAABCDEGJKKKLLRSSS94a
427,8.605440,"Wen-Syan Li, Quoc Vu, Edward Chang, Divyakant ...","Kazuyuki Shimazu, Sougata Mukherjea, Yi-Leh Wu...",International Conference on Management of Data,SIGMOD Conference,1999,1999,304578,conf/sigmod/LiVCAHMWBCHIKSS99
28614,8.339831,"Richard Thomas Snodgrass, Ilsoo Ahn, Gad Ariav...","Wolfgang Käfer, Curtis E. Dyreson, Arie Segev,...",ACM SIGMOD Record,SIGMOD Record,1994,1994,187454,journals/sigmod/SnodgrassAABCDEGJKKKLLRSSS94
7255,7.955609,"John F. Roddick, Lina Al-Jadir, Leopoldo Berto...","Kathleen Hornsby, Heidi Gregersen, Lex Wedemei...",ACM SIGMOD Record,SIGMOD Record,2000,2000,344789,journals/sigmod/RoddickABDEGHLMMMW00
25375,7.756953,"Jagannathan Srinivasan, Souripriya Das, Chuck ...","Anh-Tuan Tran, Chuck Freiwald, Aravind Yalaman...",Very Large Data Bases,VLDB,2000,2000,672004,conf/vldb/SrinivasanDFCJYKTDB00
794,7.620859,"Andrew Frank, Stephane Grumbach, Ralf Hartmut ...","Enrico Nardelli, Timos K. Sellis, Andrew U. Fr...",ACM SIGMOD Record,SIGMOD Record,1999,1999,333609,journals/sigmod/FrankGGJKLMNPSSSTW99
27871,7.578288,"Dean Daniels, Lip Boon Doo, Alan Downing, Curt...","Benny Souder, Bob Jenkins, Lip Boon Doo, Peter...",International Conference on Management of Data,SIGMOD Conference,1994,1994,191930,conf/sigmod/DanielsDDEHJJLSSS94


# **Comparing the results of our Fuzzy Matching to Ground Truth**

This segment involves comparing the data set achieved through using the fuzzymatcher with the ground truth data set provided to us. This will allow us to conclude the precision, recall and F1- meansure.

In [14]:
perfectMapping = pd.read_csv( 'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/DBLP-ACM_perfectMapping.csv')

In [15]:
perfectMapping.head()

,idDBLP,idACM
0,conf/sigmod/SlivinskasJS01,375678
1,conf/sigmod/ChaudhuriDN01,375694
2,conf/sigmod/RinfretOO01,375669
3,conf/sigmod/BreunigKKS01,375672
4,conf/sigmod/JagadishJOT01,375687


In [16]:
cols = ['DBLP_id', 'ACM_id']
fuzzy = matched_results[cols]
fuzzy.head()

,DBLP_id,ACM_id
0,conf/sigmod/VossenW99,304586
1,conf/sigmod/CruzJ99,304587
2,conf/sigmod/BouguettayaBH99,304589
3,conf/sigmod/BaruGLMPVC99,304590
4,conf/sigmod/BrodskySCE99,304582


In [17]:
## utilizing itertuples in order to compare the corresponding colums for both data sets
truePositive = 0
falsePositive = 0
falseNegative = 0

for row in fuzzy.itertuples():
  hit = perfectMapping.loc[(perfectMapping['idACM'] == row.ACM_id) & (perfectMapping['idDBLP'] == row.DBLP_id	)]
  match = hit['idDBLP'].count()

  if match == 1:
    truePositive += 1
  else: 
    falsePositive += 1

print ("truePositive: {x}".format(x=truePositive))
print ("falsePositive: {x}".format(x=falsePositive))

truePositive: 2205
falsePositive: 89


In [18]:
for row in perfectMapping.itertuples():
  hit = fuzzy.loc[(fuzzy['ACM_id'] == row.idACM) & (fuzzy['DBLP_id'] == row.idDBLP)]
  match = hit['DBLP_id'].count()
  if match == 0:
    falseNegative += 1
print ("falseNegative: {x}".format(x=falseNegative))

falseNegative: 19


# **Calculating and concluding the precision, recall and F1- meansure from the values we got through using itertuples to compare the data sets**

In [19]:
FuzzyMatcherPrecision = truePositive/(truePositive+falsePositive)
FuzzyMatcherRecall = truePositive/(truePositive+falseNegative)
FuzzyMatcherF1 = 2*((FuzzyMatcherPrecision*FuzzyMatcherRecall)/(FuzzyMatcherPrecision+FuzzyMatcherRecall))
print ("FuzzyMatcherPrecision: {x}".format(x=FuzzyMatcherPrecision))
print ("FuzzyMatcherRecall: {x}".format(x=FuzzyMatcherRecall))
print ("FuzzyMatcherF1: {x}".format(x=FuzzyMatcherF1))

FuzzyMatcherPrecision: 0.9612031386224935
FuzzyMatcherRecall: 0.9914568345323741
FuzzyMatcherF1: 0.9760956175298806


# **Utilizing RecordLinkage** 

This section involves performing performing Record Linkage with and without blocking on the two data sets and finally comparing with the ground truth set provided to confirm  precision, recall and F1- meansure.

In [20]:
import recordlinkage

In [23]:
ACM = pd.read_csv( 'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/ACM.csv', index_col='id')
DBLP = pd.read_csv( 'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/dblp.csv', index_col= 'id')
ACM.rename(columns = {'id':'ACM_id', 'title':'ACM_title', 'authors':'ACM_authors',
                              'venue':'ACM_venue', 'year':'ACM_year'}, inplace = True)
DBLP.rename(columns = {'id':'DBLP_id', 'title':'DBLP_title', 'authors':'DBLP_authors',
                              'venue':'DBLP_venue', 'year':'DBLP_year'}, inplace = True)
ACM = ACM.drop('Unnamed: 0',1)
DBLP = DBLP.drop('Unnamed: 0',1)
ACM

,ACM_title,ACM_authors,ACM_venue,ACM_year
id,,,,
304586,The WASA2 object-oriented workflow management ...,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
304587,A user-centered interface for querying distrib...,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
304589,"World Wide Database-integrating the Web, CORBA...","Athman Bouguettaya, Boualem Benatallah, Lily H...",International Conference on Management of Data,1999
304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...",International Conference on Management of Data,1999
304582,The CCUBE constraint object-oriented database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...",International Conference on Management of Data,1999
...,...,...,...,...
672977,Dual-Buffering Strategies in Object Bases,"Alfons Kemper, Donald Kossmann",Very Large Data Bases,1994
950482,Guest editorial,"Philip A. Bernstein, Yannis Ioannidis, Raghu R...",The VLDB Journal &mdash; The International Jou...,2003
672980,GraphDB: Modeling and Querying Graphs in Datab...,Ralf Hartmut G&#252;ting,Very Large Data Bases,1994


# **Full Index**

First we build the index. We will go through both full and block and compare their differenc in performance.

In [24]:
indexer = recordlinkage.Index()
indexer.full()

<Index>

In [25]:
## checking the number of matches we ant to perform
candidates = indexer.index(ACM, DBLP)
print(len(candidates))

6001104


In [26]:
## perform comparison logic
## takes 5 minutes 4s to execute using full index on my system
compare = recordlinkage.Compare()
compare.exact('ACM_year', 'DBLP_year', label='Year')
compare.string('ACM_title',
            'DBLP_title',
            threshold=0.85,
            label='Title')
compare.string('ACM_authors',
            'DBLP_authors',
            threshold=0.85,
            label='Authors')
features = compare.compute(candidates, ACM,
                        DBLP)

In [27]:
features.describe()

,Year,Title,Authors
count,6.001104e+06,6.001104e+06,6.001104e+06
mean,1.001956e-01,3.957605e-04,2.796152e-04
std,3.002606e-01,1.988980e-02,1.671936e-02
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00


In [28]:
## check the quality of matches
features.sum(axis=1).value_counts().sort_index(ascending=False)

3.0        669
2.0       1715
1.0     599900
0.0    5398820
dtype: int64

In [29]:
## taking all the records with more than 1 match as the potential matches
potential_matches = features[features.sum(axis=1) > 1].reset_index()
potential_matches['Score'] = potential_matches.loc[:, 'Year':'Title'].sum(axis=1)
potential_matches.rename(columns ={'id_1':'ACM_id','id_2':'DBLP_id'}, inplace = True)
potential_matches.head()

,ACM_id,DBLP_id,Year,Title,Authors,Score
0,304586,conf/sigmod/VossenW99,1,1.0,0.0,2.0
1,304587,conf/sigmod/CruzJ99,1,1.0,0.0,2.0
2,304589,conf/sigmod/BouguettayaBH99,1,1.0,0.0,2.0
3,304590,conf/sigmod/BaruGLMPVC99,1,1.0,0.0,2.0
4,304582,conf/sigmod/BrodskySCE99,1,1.0,0.0,2.0


# **Manually looking at a match to check the accuracy**

In [30]:
ACM.loc[304586,:]

ACM_title      The WASA2 object-oriented workflow management ...
ACM_authors                      Gottfried Vossen, Mathias Weske
ACM_venue         International Conference on Management of Data
ACM_year                                                    1999
Name: 304586, dtype: object

In [31]:
DBLP.loc["conf/sigmod/VossenW99",:]

DBLP_title      The WASA2 Object-Oriented Workflow Management ...
DBLP_authors                      Mathias Weske, Gottfried Vossen
DBLP_venue                                      SIGMOD Conference
DBLP_year                                                    1999
Name: conf/sigmod/VossenW99, dtype: object

# **Wrangling Data**

Making the data easier to review after confirming that we obtain the matches. Concatenate the data to make it wasier to look up.

In [32]:
ACM['ACM_Name_Lookup'] = ACM[[
    'ACM_authors', 'ACM_title', 'ACM_year', 'ACM_venue'
]].apply(lambda x: '_'.join(x.dropna().astype(str)), axis=1)

DBLP['DBLP_Name_Lookup'] = DBLP[[
    'DBLP_authors', 'DBLP_title', 'DBLP_year', 'DBLP_venue'
]].apply(lambda x: '_'.join(x.dropna().astype(str)), axis=1)

ACM_lookup = ACM[['ACM_Name_Lookup']].reset_index()
ACM_lookup.rename(columns ={'id':'ACM_id'}, inplace = True)
DBLP_lookup = DBLP[['DBLP_Name_Lookup']].reset_index()
DBLP_lookup.rename(columns ={'id':'DBLP_id'}, inplace = True)


In [33]:
ACM_lookup.head()

,ACM_id,ACM_Name_Lookup
0,304586,"Gottfried Vossen, Mathias Weske_The WASA2 obje..."
1,304587,"Isabel F. Cruz, Kimberly M. James_A user-cente..."
2,304589,"Athman Bouguettaya, Boualem Benatallah, Lily H..."
3,304590,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228..."
4,304582,"Alexander Brodsky, Victor E. Segal, Jia Chen, ..."


In [34]:
DBLP_lookup.head()

,DBLP_id,DBLP_Name_Lookup
0,journals/sigmod/Mackay99,D. Scott Mackay_Semantic Integration of Enviro...
1,conf/vldb/PoosalaI96,"Viswanath Poosala, Yannis E. Ioannidis_Estimat..."
2,conf/vldb/PalpanasSCP02,"Themistoklis Palpanas, Richard Sidle, Hamid Pi..."
3,conf/vldb/GardarinGT96,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert G..."
4,conf/vldb/HoelS95,"Erik G. Hoel, Hanan Samet_Benchmarking Spatial..."


In [35]:
## merge with the ACM data
ACM_merge = potential_matches.merge(ACM_lookup, how='left')
ACM_merge.head()

,ACM_id,DBLP_id,Year,Title,Authors,Score,ACM_Name_Lookup
0,304586,conf/sigmod/VossenW99,1,1.0,0.0,2.0,"Gottfried Vossen, Mathias Weske_The WASA2 obje..."
1,304587,conf/sigmod/CruzJ99,1,1.0,0.0,2.0,"Isabel F. Cruz, Kimberly M. James_A user-cente..."
2,304589,conf/sigmod/BouguettayaBH99,1,1.0,0.0,2.0,"Athman Bouguettaya, Boualem Benatallah, Lily H..."
3,304590,conf/sigmod/BaruGLMPVC99,1,1.0,0.0,2.0,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228..."
4,304582,conf/sigmod/BrodskySCE99,1,1.0,0.0,2.0,"Alexander Brodsky, Victor E. Segal, Jia Chen, ..."


In [36]:
##final merge with the DBLP data
final_merge = ACM_merge.merge(DBLP_lookup, how='left')

In [37]:
cols = ['ACM_id', 'DBLP_id', 'Score',
        'ACM_Name_Lookup', 'DBLP_Name_Lookup']

final_merge[cols].sort_values(by=['ACM_id', 'Score'], ascending=False)

,ACM_id,DBLP_id,Score,ACM_Name_Lookup,DBLP_Name_Lookup
2220,959087,journals/sigmod/Hainaut03,2.0,Jean-Luc Hainaut_Research in database engineer...,Jean-Luc Hainaut_Research in database engineer...
2219,959086,journals/sigmod/KambhampatiK03,2.0,"Subbarao Kambhampati, Craig A. Knoblock_Inform...","Subbarao Kambhampati, Craig A. Knoblock_Inform..."
2222,959085,journals/sigmod/MotroA03,2.0,"Amihai Motro, Troels Andreasen_Report on FQAS ...","Amihai Motro, Troels Andreasen_Report on FQAS ..."
2221,959084,journals/sigmod/Giorgini03,2.0,Paolo Giorgini_Agent-Oriented software enginee...,Paolo Giorgini_Agent-Oriented software enginee...
2292,959083,journals/sigmod/LenzVJS03,2.0,"Hans J. Lenz, Panos Vassiliadis, Manfred Jeusf...","Manfred A. Jeusfeld, Hans-Joachim Lenz, Panos ..."
...,...,...,...,...,...
1377,176568,journals/tods/RosenthalR94,2.0,"Arnon Rosenthal, David Reiner_Tools and transf...","David S. Reiner, Arnon Rosenthal_Tools and Tra..."
1375,174642,journals/tods/CliffordC94,2.0,"James Clifford, Albert Croker, Alexander Tuzhi...","James Clifford, Albert Croker, Alexander Tuzhi..."
1395,174641,journals/tods/TendickM94,2.0,"Patrick Tendick, Norman Matloff_A modified ran...","Norman S. Matloff, Patrick Tendick_A Modified ..."
1393,174640,journals/tods/OliverS94,2.0,"Martin S. Olivier, Sebastiaan H. von Solms_A t...","Sebastiaan H. von Solms, Martin S. Olivier_A T..."


# **Comparing the results of our full Index Record Linkage to Ground Truth**

This segment involves comparing the data set achieved through using the full index Record Linkage with the ground truth data set provided to us. This will allow us to conclude the precision, recall and F1- meansure.**

In [38]:
cols = ['ACM_id','DBLP_id']
recordLinkage_NB = final_merge[cols]
recordLinkage_NB.head()

,ACM_id,DBLP_id
0,304586,conf/sigmod/VossenW99
1,304587,conf/sigmod/CruzJ99
2,304589,conf/sigmod/BouguettayaBH99
3,304590,conf/sigmod/BaruGLMPVC99
4,304582,conf/sigmod/BrodskySCE99


In [39]:
## utilizing itertuples in order to compare the corresponding colums for both data sets
truePositive = 0
falsePositive = 0
falseNegative = 0

for row in recordLinkage_NB.itertuples():
  hit = perfectMapping.loc[(perfectMapping['idACM'] == row.ACM_id) & (perfectMapping['idDBLP'] == row.DBLP_id	)]
  match = hit['idDBLP'].count()

  if match == 1:
    truePositive += 1
  else: 
    falsePositive += 1

print ("truePositive: {x}".format(x=truePositive))
print ("falsePositive: {x}".format(x=falsePositive))

truePositive: 2132
falsePositive: 252


In [40]:
for row in perfectMapping.itertuples():
  hit = recordLinkage_NB.loc[(recordLinkage_NB['ACM_id'] == row.idACM) & (recordLinkage_NB['DBLP_id'] == row.idDBLP)]
  match = hit['DBLP_id'].count()
  if match == 0:
    falseNegative += 1
print ("falseNegative: {x}".format(x=falseNegative))

falseNegative: 92


# **Calculating and concluding the precision, recall and F1- meansure from the values we got through using itertuples to compare the data sets**

In [41]:
recordLinkage_NBPrecision = truePositive/(truePositive+falsePositive)
recordLinkage_NBRecall = truePositive/(truePositive+falseNegative)
recordLinkage_NBF1 = 2*((recordLinkage_NBPrecision*recordLinkage_NBRecall)/(recordLinkage_NBPrecision+recordLinkage_NBRecall))
print ("recordLinkage_NBPrecision: {x}".format(x=recordLinkage_NBPrecision))
print ("recordLinkage_NBRecall: {x}".format(x=recordLinkage_NBRecall))
print ("recordLinkage_NBF1: {x}".format(x=recordLinkage_NBF1))

recordLinkage_NBPrecision: 0.8942953020134228
recordLinkage_NBRecall: 0.9586330935251799
recordLinkage_NBF1: 0.9253472222222222


# **Blocking**

now we build an index using blocking to compare the difference in performance to the record linkage with full index. We choose to block the year feature and not consider matches with different years. This results in a significant upgrade in performance, causing the processing time to greatly decrease. later on, it will aslo be revealed that the there is no drop in quality of matches as a consequence to the increase in performance.

In [42]:
indexer = recordlinkage.Index()
indexer.block(left_on='ACM_year', right_on='DBLP_year')
## checking the number of matches we ant to perform
candidates = indexer.index(ACM, DBLP)
print(len(candidates))

601284


In [43]:
## perform comparison logic
## takes 38s to execute using blocked index on my system
compare = recordlinkage.Compare()
compare.string('ACM_title',
            'DBLP_title',
            threshold=0.85,
            label='Title')
compare.string('ACM_authors',
            'DBLP_authors',
            threshold=0.85,
            label='Authors')
features = compare.compute(candidates, ACM,
                        DBLP)

**Features**

Changed the required matching features from greater than 1 to greater than zero as we can consider the blocked feature as one of the features as data without matching years will not be consider either ways. This was originally kept the same as before but changed to to the resulting upsurge of false negatives.

In [44]:
potential_matches = features[features.sum(axis=1) > 0].reset_index()
potential_matches['Score'] = potential_matches.loc[:, 'Authors':'Title'].sum(axis=1)
potential_matches.rename(columns ={'id_1':'ACM_id','id_2':'DBLP_id'}, inplace = True)
potential_matches.head()

,ACM_id,DBLP_id,Title,Authors,Score
0,304586,conf/sigmod/VossenW99,1.0,0.0,0.0
1,304587,conf/sigmod/CruzJ99,1.0,0.0,0.0
2,304589,conf/sigmod/BouguettayaBH99,1.0,0.0,0.0
3,304590,conf/sigmod/BaruGLMPVC99,1.0,0.0,0.0
4,304582,conf/sigmod/BrodskySCE99,1.0,0.0,0.0


In [45]:
ACM['ACM_Name_Lookup'] = ACM[[
    'ACM_authors', 'ACM_title', 'ACM_year', 'ACM_venue'
]].apply(lambda x: '_'.join(x.dropna().astype(str)), axis=1)

DBLP['DBLP_Name_Lookup'] = DBLP[[
    'DBLP_authors', 'DBLP_title', 'DBLP_year', 'DBLP_venue'
]].apply(lambda x: '_'.join(x.dropna().astype(str)), axis=1)

ACM_lookup = ACM[['ACM_Name_Lookup']].reset_index()
ACM_lookup.rename(columns ={'id':'ACM_id'}, inplace = True)
DBLP_lookup = DBLP[['DBLP_Name_Lookup']].reset_index()
DBLP_lookup.rename(columns ={'id':'DBLP_id'}, inplace = True)

In [46]:
## merge with the ACM data
ACM_merge = potential_matches.merge(ACM_lookup, how='left')
ACM_merge.head()

,ACM_id,DBLP_id,Title,Authors,Score,ACM_Name_Lookup
0,304586,conf/sigmod/VossenW99,1.0,0.0,0.0,"Gottfried Vossen, Mathias Weske_The WASA2 obje..."
1,304587,conf/sigmod/CruzJ99,1.0,0.0,0.0,"Isabel F. Cruz, Kimberly M. James_A user-cente..."
2,304589,conf/sigmod/BouguettayaBH99,1.0,0.0,0.0,"Athman Bouguettaya, Boualem Benatallah, Lily H..."
3,304590,conf/sigmod/BaruGLMPVC99,1.0,0.0,0.0,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228..."
4,304582,conf/sigmod/BrodskySCE99,1.0,0.0,0.0,"Alexander Brodsky, Victor E. Segal, Jia Chen, ..."


In [47]:
##final merge with the DBLP data
final_merge_B = ACM_merge.merge(DBLP_lookup, how='left')

In [48]:
cols = ['ACM_id', 'DBLP_id', 'Score',
        'ACM_Name_Lookup', 'DBLP_Name_Lookup']

final_merge_B[cols].sort_values(by=['ACM_id', 'Score'], ascending=False)

,ACM_id,DBLP_id,Score,ACM_Name_Lookup,DBLP_Name_Lookup
2262,959087,journals/sigmod/Hainaut03,0.0,Jean-Luc Hainaut_Research in database engineer...,Jean-Luc Hainaut_Research in database engineer...
2261,959086,journals/sigmod/KambhampatiK03,0.0,"Subbarao Kambhampati, Craig A. Knoblock_Inform...","Subbarao Kambhampati, Craig A. Knoblock_Inform..."
2264,959085,journals/sigmod/MotroA03,0.0,"Amihai Motro, Troels Andreasen_Report on FQAS ...","Amihai Motro, Troels Andreasen_Report on FQAS ..."
2263,959084,journals/sigmod/Giorgini03,0.0,Paolo Giorgini_Agent-Oriented software enginee...,Paolo Giorgini_Agent-Oriented software enginee...
2295,959083,journals/sigmod/LenzVJS03,0.0,"Hans J. Lenz, Panos Vassiliadis, Manfred Jeusf...","Manfred A. Jeusfeld, Hans-Joachim Lenz, Panos ..."
...,...,...,...,...,...
1919,176568,journals/tods/RosenthalR94,0.0,"Arnon Rosenthal, David Reiner_Tools and transf...","David S. Reiner, Arnon Rosenthal_Tools and Tra..."
1918,174642,journals/tods/CliffordC94,0.0,"James Clifford, Albert Croker, Alexander Tuzhi...","James Clifford, Albert Croker, Alexander Tuzhi..."
1927,174641,journals/tods/TendickM94,0.0,"Patrick Tendick, Norman Matloff_A modified ran...","Norman S. Matloff, Patrick Tendick_A Modified ..."
1926,174640,journals/tods/OliverS94,0.0,"Martin S. Olivier, Sebastiaan H. von Solms_A t...","Sebastiaan H. von Solms, Martin S. Olivier_A T..."


# **Comparing the results of our Blocked Index Record Linkage to Ground Truth**

This segment involves comparing the data set achieved through using the Blocked Index Record Linkage with the ground truth data set provided to us. This will allow us to conclude the precision, recall and F1- meansure.

In [49]:
cols = ['ACM_id','DBLP_id']
recordLinkage_B = final_merge_B[cols]
recordLinkage_B.head()

,ACM_id,DBLP_id
0,304586,conf/sigmod/VossenW99
1,304587,conf/sigmod/CruzJ99
2,304589,conf/sigmod/BouguettayaBH99
3,304590,conf/sigmod/BaruGLMPVC99
4,304582,conf/sigmod/BrodskySCE99


In [50]:
## utilizing itertuples in order to compare the corresponding colums for both data sets
truePositive = 0
falsePositive = 0
falseNegative = 0

for row in recordLinkage_B.itertuples():
  hit = perfectMapping.loc[(perfectMapping['idACM'] == row.ACM_id) & (perfectMapping['idDBLP'] == row.DBLP_id	)]
  match = hit['idDBLP'].count()

  if match == 1:
    truePositive += 1
  else: 
    falsePositive += 1

print ("truePositive: {x}".format(x=truePositive))
print ("falsePositive: {x}".format(x=falsePositive))

truePositive: 2132
falsePositive: 213


In [51]:
for row in perfectMapping.itertuples():
  hit = recordLinkage_B.loc[(recordLinkage_B['ACM_id'] == row.idACM) & (recordLinkage_B['DBLP_id'] == row.idDBLP)]
  match = hit['DBLP_id'].count()
  if match == 0:
    falseNegative += 1
print ("falseNegative: {x}".format(x=falseNegative))

falseNegative: 92


# **Calculating and concluding the precision, recall and F1- meansure from the values we got through using itertuples to compare the data sets**

In [52]:
recordLinkage_BPrecision = truePositive/(truePositive+falsePositive)
recordLinkage_BRecall = truePositive/(truePositive+falseNegative)
recordLinkage_BF1 = 2*((recordLinkage_BPrecision*recordLinkage_BRecall)/(recordLinkage_BPrecision+recordLinkage_BRecall))
print ("recordLinkage_BPrecision: {x}".format(x=recordLinkage_BPrecision))
print ("recordLinkage_BRecall: {x}".format(x=recordLinkage_BRecall))
print ("recordLinkage_BF1: {x}".format(x=recordLinkage_BF1))

recordLinkage_BPrecision: 0.9091684434968017
recordLinkage_BRecall: 0.9586330935251799
recordLinkage_BF1: 0.9332457868242504


# **Attempting to improve results**
Attempting to change some setting in the full Index record linkage in order to obtain better results. I played around with threshold parameters in order to attempt to get better results.

In [54]:
ACM = pd.read_csv( 'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/ACM.csv', index_col='id')
DBLP = pd.read_csv( 'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/dblp.csv', index_col= 'id')
ACM.rename(columns = {'id':'ACM_id', 'title':'ACM_title', 'authors':'ACM_authors',
                              'venue':'ACM_venue', 'year':'ACM_year'}, inplace = True)
DBLP.rename(columns = {'id':'DBLP_id', 'title':'DBLP_title', 'authors':'DBLP_authors',
                              'venue':'DBLP_venue', 'year':'DBLP_year'}, inplace = True)
ACM = ACM.drop('Unnamed: 0',1)
DBLP = DBLP.drop('Unnamed: 0',1)
ACM

,ACM_title,ACM_authors,ACM_venue,ACM_year
id,,,,
304586,The WASA2 object-oriented workflow management ...,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
304587,A user-centered interface for querying distrib...,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
304589,"World Wide Database-integrating the Web, CORBA...","Athman Bouguettaya, Boualem Benatallah, Lily H...",International Conference on Management of Data,1999
304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...",International Conference on Management of Data,1999
304582,The CCUBE constraint object-oriented database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...",International Conference on Management of Data,1999
...,...,...,...,...
672977,Dual-Buffering Strategies in Object Bases,"Alfons Kemper, Donald Kossmann",Very Large Data Bases,1994
950482,Guest editorial,"Philip A. Bernstein, Yannis Ioannidis, Raghu R...",The VLDB Journal &mdash; The International Jou...,2003
672980,GraphDB: Modeling and Querying Graphs in Datab...,Ralf Hartmut G&#252;ting,Very Large Data Bases,1994


In [55]:
indexer = recordlinkage.Index()
indexer.full()
candidates = indexer.index(ACM, DBLP)

threshold value for the title comparison is lowered from 0.85 to 0.75 to reduce the number of false negatives, and produce better values for precision, recall and f1 measure.

In [56]:
compare = recordlinkage.Compare()
compare.exact('ACM_year', 'DBLP_year', label='Year')
## changed the threshold from 0.85 to 0.75
compare.string('ACM_title',
            'DBLP_title',
            threshold=0.75,
            label='Title')
compare.string('ACM_authors',
            'DBLP_authors',
            threshold=0.85,
            label='Authors')
features = compare.compute(candidates, ACM,
                        DBLP)
potential_matches = features[features.sum(axis=1) > 1].reset_index()
potential_matches['Score'] = potential_matches.loc[:, 'Year':'Title'].sum(axis=1)
potential_matches.rename(columns ={'id_1':'ACM_id','id_2':'DBLP_id'}, inplace = True)
potential_matches.head()

,ACM_id,DBLP_id,Year,Title,Authors,Score
0,304586,conf/sigmod/VossenW99,1,1.0,0.0,2.0
1,304587,conf/sigmod/CruzJ99,1,1.0,0.0,2.0
2,304589,conf/sigmod/BouguettayaBH99,1,1.0,0.0,2.0
3,304590,conf/sigmod/BaruGLMPVC99,1,1.0,0.0,2.0
4,304582,conf/sigmod/BrodskySCE99,1,1.0,0.0,2.0


In [57]:
ACM['ACM_Name_Lookup'] = ACM[[
    'ACM_authors', 'ACM_title', 'ACM_year', 'ACM_venue'
]].apply(lambda x: '_'.join(x.dropna().astype(str)), axis=1)

DBLP['DBLP_Name_Lookup'] = DBLP[[
    'DBLP_authors', 'DBLP_title', 'DBLP_year', 'DBLP_venue'
]].apply(lambda x: '_'.join(x.dropna().astype(str)), axis=1)

ACM_lookup = ACM[['ACM_Name_Lookup']].reset_index()
ACM_lookup.rename(columns ={'id':'ACM_id'}, inplace = True)
DBLP_lookup = DBLP[['DBLP_Name_Lookup']].reset_index()
DBLP_lookup.rename(columns ={'id':'DBLP_id'}, inplace = True)

In [58]:
ACM_merge = potential_matches.merge(ACM_lookup, how='left')
final_merge = ACM_merge.merge(DBLP_lookup, how='left')

In [59]:
cols = ['ACM_id','DBLP_id']
recordLinkage_NB_new = final_merge[cols]

truePositive = 0
falsePositive = 0
falseNegative = 0

for row in recordLinkage_NB_new.itertuples():
  hit = perfectMapping.loc[(perfectMapping['idACM'] == row.ACM_id) & (perfectMapping['idDBLP'] == row.DBLP_id	)]
  match = hit['idDBLP'].count()

  if match == 1:
    truePositive += 1
  else: 
    falsePositive += 1

for row in perfectMapping.itertuples():
  hit = recordLinkage_NB_new.loc[(recordLinkage_NB_new['ACM_id'] == row.idACM) & (recordLinkage_NB_new['DBLP_id'] == row.idDBLP)]
  match = hit['DBLP_id'].count()
  if match == 0:
    falseNegative += 1

print ("truePositive: {x}".format(x=truePositive))
print ("falsePositive: {x}".format(x=falsePositive))
print ("falseNegative: {x}".format(x=falseNegative))

truePositive: 2176
falsePositive: 259
falseNegative: 48


In [61]:
recordLinkage_NB_newPrecision = truePositive/(truePositive+falsePositive)
recordLinkage_NB_newRecall = truePositive/(truePositive+falseNegative)
recordLinkage_NB_newF1 = 2*((recordLinkage_NB_newPrecision*recordLinkage_NB_newRecall)/(recordLinkage_NB_newPrecision+recordLinkage_NB_newRecall))
print ("recordLinkage_NB_newPrecision: {x}".format(x=recordLinkage_NB_newPrecision))
print ("recordLinkage_NB_newRecall: {x}".format(x=recordLinkage_NB_newRecall))
print ("recordLinkage_NB_newF1: {x}".format(x=recordLinkage_NB_newF1))

recordLinkage_NB_newPrecision: 0.8936344969199178
recordLinkage_NB_newRecall: 0.9784172661870504
recordLinkage_NB_newF1: 0.9341060313371968


# **Comparison of Results**

Below we can see the results of all three of our Record Linkage implementations along with the fuzzymatcher implementation. We can take note that the modified full index implementation has higher values for both Recall and F1 measure. This can be acquited to the lowering of the threshold of the comparison of titles slightly. It is also important to take note that the record linkage using blocking has equally high scores as the new full index implementation, while being significanlty faster to process. 

Overall it can be noted that the fuzzy matching implementation has overall higher precision, recall and f1 measure scores than the record linkage implemtnations, accompanied with being faster to implement. Though, this might also be due to this particular data set being more suitable for this particular implementation. More could have explored regarding the feature to be blocked, as well as the implementation of the sorted neighbourhood algorythm.

In [62]:
print ("FuzzyMatcherPrecision: {x}".format(x=FuzzyMatcherPrecision))
print ("FuzzyMatcherRecall: {x}".format(x=FuzzyMatcherRecall))
print ("FuzzyMatcherF1: {x}".format(x=FuzzyMatcherF1))
print("\n")
print ("recordLinkage_NBPrecision: {x}".format(x=recordLinkage_NBPrecision))
print ("recordLinkage_NBRecall: {x}".format(x=recordLinkage_NBRecall))
print ("recordLinkage_NBF1: {x}".format(x=recordLinkage_NBF1))
print("\n")
print ("recordLinkage_BPrecision: {x}".format(x=recordLinkage_BPrecision))
print ("recordLinkage_BRecall: {x}".format(x=recordLinkage_BRecall))
print ("recordLinkage_BF1: {x}".format(x=recordLinkage_BF1))
print("\n")
print ("recordLinkage_NB_newPrecision: {x}".format(x=recordLinkage_NB_newPrecision))
print ("recordLinkage_NB_newRecall: {x}".format(x=recordLinkage_NB_newRecall))
print ("recordLinkage_NB_newF1: {x}".format(x=recordLinkage_NB_newF1))

FuzzyMatcherPrecision: 0.9612031386224935
FuzzyMatcherRecall: 0.9914568345323741
FuzzyMatcherF1: 0.9760956175298806


recordLinkage_NBPrecision: 0.8942953020134228
recordLinkage_NBRecall: 0.9586330935251799
recordLinkage_NBF1: 0.9253472222222222


recordLinkage_BPrecision: 0.9091684434968017
recordLinkage_BRecall: 0.9586330935251799
recordLinkage_BF1: 0.9332457868242504


recordLinkage_NB_newPrecision: 0.8936344969199178
recordLinkage_NB_newRecall: 0.9784172661870504
recordLinkage_NB_newF1: 0.9341060313371968
